# IMPORTS

In [113]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from torchvision import transforms
from tqdm.auto import tqdm
from torch.autograd import Variable

# Hyperparameters

In [114]:
# Hyperparameters for LSTM

IN_DIM = 2 
SEQUENCE_LENGTH = 1  # Lookback window 
LSTM_IN_DIM = int(IN_DIM / SEQUENCE_LENGTH)  # The input size of LSTM is equal to the total variable length/time series length
LSTM_HIDDEN_DIM = 300  # LSTM Hidden state
OUT_DIM = 1  # Output size
LEARNING_RATE = 0.01  # learning rate
WEIGHT_DECAY = 1e-6  # L2 Penalty
BATCH_SIZE = 50  # batch size
EPOCHS = 2000  # epoch Size
TRAIN_PER = 0.80  # Proportion of training set
VALI_PER = 0.0  # Proportion of validation set
USE_GPU = False

In [115]:
# Hyperparameters for ANN
num_epochs = 2000
input_size = 2
hidden_size = 2
num_layers = 1
num_classes = 1
#net = ANN(input_size,20,num_classes)

# DATASET PROCESSING

## Dataset Analysis

In [116]:
EXCEL_PATH = 'Data Mining.xlsx'
EPOCHS = 200
LEARNING_RATE = 0.01
RANDOM_SEED = 42

df = pd.read_excel(EXCEL_PATH)
df

,DATE,SW273,SW267,SW269
0,1979-06-01,2.60,5.52,5.42
1,1979-06-02,2.43,5.30,5.24
2,1979-06-03,2.40,5.07,5.09
3,1979-06-04,2.29,4.87,4.94
4,1979-06-05,2.20,4.69,4.79
...,...,...,...,...
5010,2009-10-27,3.50,6.13,5.59
5011,2009-10-28,3.34,5.97,5.50
5012,2009-10-29,3.20,5.82,5.42
5013,2009-10-30,3.12,5.70,5.36


## Custom Dataset Class

In [117]:
class FloodDataset(Dataset):
  """Custom Dataset class to work with FloodDataset"""
  def __init__(self, excel_path, train, items=None):
    df = pd.read_excel(excel_path)
    
    if items and train==True:
      df = df[:items]

    elif items and train==False:
      df = df[-items:]

    df['index'] = range(0, len(df))
    df = df.set_index('index')
    
    self.excel_path = excel_path
    self.features = df[['SW273','SW269']]
    self.targets = df[['SW267']]

  def __getitem__(self, index):
    return torch.tensor(self.features.iloc[index], dtype=torch.float32), torch.tensor(self.targets.iloc[index], dtype=torch.float32)

  def __len__(self):
    return len(self.features)

In [118]:
# df[['SW267', 'SW269']].iloc[0]

In [119]:
df.describe()

,SW273,SW267,SW269
count,5015.000000,5015.000000,5015.000000
mean,4.971986,8.797886,7.088640
std,1.248850,2.063672,1.233145
min,1.670000,3.140000,3.180000
25%,4.050000,7.730000,6.460000
50%,5.300000,9.510000,7.430000
75%,5.900000,10.340000,7.970000
max,7.780000,12.440000,9.750000


In [120]:
df.SW273.isnull().sum(), df.SW267.isnull().sum(), df.SW269.isnull().sum()

(0, 0, 0)

## Dataset and Dataloader

In [121]:
train_dataset = FloodDataset(excel_path=EXCEL_PATH,
                             train=True,
                             items = 4011)
test_dataset = FloodDataset(excel_path=EXCEL_PATH,
                             train=False,
                             items = 1003)
print(f'Length Train Dataset: {len(train_dataset)} \nLength of Test Dataset: {len(test_dataset)}')

train_dataloader = DataLoader(dataset=train_dataset,
                          batch_size = 1000,
                          shuffle=False,
                          num_workers = 0)
test_dataloader = DataLoader(dataset=test_dataset,
                         batch_size = 1003,
                         shuffle=False,
                         num_workers=0)

print(f'Length Train Loader: {len(train_dataloader)} \nLength of Test Loader: {len(test_dataloader)}')

Length Train Dataset: 4011 
Length of Test Dataset: 1003
Length Train Loader: 5 
Length of Test Loader: 1


In [122]:
# train_dataset.input, train_dataset.output
# test_dataset.input, test_dataset.output

In [123]:
# sample = next(iter(train_loader))
# print(len(sample))
# inp, out = sample
# print(inp.shape, out.shape)
# type(inp) ,type(out)

# MODELS

## LSTM

In [124]:
class LSTM(nn.Module):
    
    def __init__(self,
                 in_dim,
                 sequence_length,
                 lstm_in_dim,
                 lstm_hidden_dim,
                 out_dim,
                 use_gpu=False):

        super(LSTM,self).__init__()

        # Parameter import part
        self.in_dim = in_dim
        self.sequence_length = sequence_length
        self.lstm_in_dim = lstm_in_dim
        self.lstm_hidden_dim = lstm_hidden_dim
        self.out_dim = out_dim
        self.use_gpu = use_gpu

        # batch_norm layer
        self.batch_norm = nn.BatchNorm1d(in_dim)
        
        # input layer
        self.layer_in = nn.Linear(in_dim, in_dim)
        
        # lstmcell 
        self.lstmcell = nn.LSTMCell(lstm_in_dim, lstm_hidden_dim)
         
        # output layer, 1 × lstm_hiddendim -> 1 × 1
        self.layer_out = nn.Linear(lstm_hidden_dim, out_dim,bias=False)
        
        # activate functions
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

    def forward (self,input):
        out = self.batch_norm(input)
        out = self.layer_in(out)

        # Initialize hidden state and memory cell state
        h_t_1 = torch.zeros(out.size(0), self.lstm_hidden_dim) # batch, hideden_size
        c_t_1 = torch.zeros(out.size(0), self.lstm_hidden_dim) # batch, hideden_size
       
        for i in range(self.sequence_length):
            x_t = out[:,i*self.lstm_in_dim:(i+1)*(self.lstm_in_dim)]
            h_t,c_t = self.lstmcell(x_t,(h_t_1,c_t_1)) 
            h_t_1,c_t_1 = h_t,c_t
        out = self.layer_out(h_t)
        return out

## Spatio-LSTM

In [125]:
class SA_LSTM(nn.Module):

    def __init__(self, in_dim, sequence_length, lstm_in_dim, lstm_hidden_dim, out_dim, use_gpu=False):
        super(SA_LSTM,self).__init__()
 
        self.in_dim = in_dim
        self.sequence_length = sequence_length
        self.lstm_in_dim = lstm_in_dim
        self.lstm_hidden_dim = lstm_hidden_dim
        self.out_dim = out_dim
        self.use_gpu = use_gpu
        
        self.batch_norm = nn.BatchNorm1d(in_dim)

        self.layer_in = nn.Linear(in_dim, in_dim,bias=False)


        self.S_A = nn.Linear(lstm_in_dim, lstm_in_dim)

        self.lstmcell = nn.LSTMCell(lstm_in_dim, lstm_hidden_dim)

        # # output layer, 1 × lstm_hiddendim -> 1 × 1
        self.layer_out = nn.Linear(lstm_hidden_dim, out_dim,bias=False)
         
        # activate functions
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward (self,input):

        out = self.batch_norm(input)
        out = self.layer_in(out)
        h_t_1 = torch.zeros(out.size(0), self.lstm_hidden_dim) 
        c_t_1 = torch.zeros(out.size(0), self.lstm_hidden_dim)
      
        for i in range(self.sequence_length):
            x_t = out[       : ,       i*self.lstm_in_dim : (i+1)*(self.lstm_in_dim)]
            alpha_t =  self.sigmoid(self.S_A(x_t))
            alpha_t = self.softmax(alpha_t)
            h_t,c_t = self.lstmcell(x_t*alpha_t,(h_t_1,c_t_1))
            h_t_1,c_t_1 = h_t,c_t
        out = self.layer_out(h_t)
        return out

## Temporal-LSTM

In [126]:
class TA_LSTM(nn.Module):

    def __init__(self, in_dim, sequence_length, lstm_in_dim, lstm_hidden_dim, out_dim, use_gpu=False):
        super(TA_LSTM,self).__init__()
        
        self.in_dim = in_dim
        self.sequence_length = sequence_length
        self.lstm_in_dim = lstm_in_dim
        self.lstm_hidden_dim = lstm_hidden_dim
        self.out_dim = out_dim
        self.use_gpu = use_gpu
        
        self.batch_norm = nn.BatchNorm1d(in_dim)
        
        self.layer_in = nn.Linear(in_dim, in_dim,bias=False)
        # lstmcell 
        self.lstmcell = nn.LSTMCell(lstm_in_dim, lstm_hidden_dim)
        # temporal attention module, 
        self.T_A = nn.Linear(sequence_length*lstm_hidden_dim, sequence_length)
        # # output layer, 
        self.layer_out = nn.Linear(lstm_hidden_dim, out_dim,bias=False)
        # activate functions
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward (self,input):

        out = self.batch_norm(input)
        out = self.layer_in(out)
        h_t_1 = torch.zeros(out.size(0), self.lstm_hidden_dim) # batch, hidden_size
        c_t_1 = torch.zeros(out.size(0), self.lstm_hidden_dim) # batch, hidden_size
      
        h_list = []

        for i in range(self.sequence_length):
            
            x_t = out[:,i*self.lstm_in_dim:(i+1)*(self.lstm_in_dim)]
           
            h_t,c_t = self.lstmcell(x_t,(h_t_1,c_t_1)) 
            
            h_list.append(h_t)

            h_t_1,c_t_1 = h_t,c_t
        
        total_ht = h_list[0]
        for i in range(1,len(h_list)):
            total_ht = torch.cat((total_ht,h_list[i]),1)    
        
        beta_t =  self.relu(self.T_A(total_ht))
        beta_t = self.softmax(beta_t)
        
        out = torch.zeros(out.size(0), self.lstm_hidden_dim)
        # print(h_list[i].size(),beta_t[:,1].size())

        for i in range(len(h_list)):
                      
            out = out + h_list[i]*beta_t[:,i].reshape(out.size(0),1)

        out = self.layer_out(out)

        return out

## Spatio-Temporal LSTM

In [127]:
class STA_LSTM(nn.Module):

    def __init__(self,
                 in_dim,
                 sequence_length,
                 lstm_in_dim,
                 lstm_hidden_dim,
                 out_dim,
                 use_gpu=False):

        super(STA_LSTM,self).__init__()

        self.in_dim = in_dim
        self.sequence_length = sequence_length
        
        self.lstm_in_dim = lstm_in_dim
        self.lstm_hidden_dim = lstm_hidden_dim
        self.out_dim = out_dim
        self.use_gpu = use_gpu

        # batch_norm layer
        self.batch_norm = nn.BatchNorm1d(in_dim)
        
        # input layer
        self.layer_in = nn.Linear(in_dim, in_dim,bias=False)

        # spatial attention module
        self.S_A = nn.Linear(lstm_in_dim, lstm_in_dim)
        
        # lstmcell 
        self.lstmcell = nn.LSTMCell(lstm_in_dim, lstm_hidden_dim)
        
        # temporal atteention module
        self.T_A = nn.Linear(sequence_length*lstm_hidden_dim, sequence_length)
        
        # # output layer, 1 × lstm_hiddendim -> 1 × 1
        self.layer_out = nn.Linear(lstm_hidden_dim, out_dim,bias=False)
         
        # activate functions
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward (self,input):

        out = self.batch_norm(input)

        out = self.layer_in(out)
         
        h_t_1 = torch.zeros(out.size(0), self.lstm_hidden_dim) # batch, hidden_size
        c_t_1 = torch.zeros(out.size(0), self.lstm_hidden_dim) # batch, hidden_size
      
        h_list = []

        for i in range(self.sequence_length):
            
            x_t = out[: ,  i*self.lstm_in_dim  :      (i+1)*(self.lstm_in_dim)]
           
            alpha_t =  self.sigmoid(self.S_A(x_t))

            alpha_t = self.softmax(alpha_t)
            # print(alpha_t)

            h_t,c_t = self.lstmcell(x_t*alpha_t,(h_t_1,c_t_1)) 
            
            h_list.append(h_t)

            h_t_1,c_t_1 = h_t,c_t
        
        total_ht = h_list[0]
        for i in range(1,len(h_list)):
            total_ht = torch.cat((total_ht,h_list[i]),1)    
        
        beta_t =  self.relu(self.T_A(total_ht))
        beta_t = self.softmax(beta_t)
        # print(beta_t)
        out = torch.zeros(out.size(0), self.lstm_hidden_dim)
        # print(h_list[i].size(),beta_t[:,1].size())

        for i in range(len(h_list)):
                      
            out = out + h_list[i]*beta_t[:,i].reshape(out.size(0),1)

        out = self.layer_out(out)
        
        return out

## ANN

In [128]:
class ANN(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(ANN, self).__init__()
    self.fc1 = nn.Linear(in_features=input_size, out_features=hidden_size)
    self.relu = nn.ReLU()
    self.output_layer = nn.Linear(in_features=hidden_size, out_features=num_classes)

  def forward(self, x):
#     x = x.reshape(x.size(0), 1)
    x = self.fc1(x)
    x = self.relu(x)
    x = self.output_layer(x)
    return x

# **Model Preparation**

In [129]:
net= ANN(IN_DIM,5,OUT_DIM)
net = STA_LSTM(IN_DIM, SEQUENCE_LENGTH, LSTM_IN_DIM, LSTM_HIDDEN_DIM, OUT_DIM, USE_GPU)
print('The network model is ready')
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
adjust_lr = optim.lr_scheduler.MultiStepLR(optimizer,  milestones=[i * 10 for i in range(EPOCHES // 10)], gamma=0.5)

The network model is ready


# Evaluation Metrics Preparation

### Evaluation Metric Classes(RMSE, R2)
These cells are not being used at the moment

In [130]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
    def forward(self,yhat,y):
        return torch.sqrt(self.mse(yhat,y))
#criterion = RMSELoss()     #loss = criterion(yhat,y)

In [131]:
def r2_loss(output, target):
    target_mean = torch.mean(target)
    ss_tot = torch.sum((target - target_mean) ** 2)
    ss_res = torch.sum((target - output) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

### Loss and Calculate Functions

In [132]:
def Loss(preds, targets, train):
    error = preds-targets
    abs_error = torch.abs(error)
    
    mse = torch.sum(error**2)
    rmse = torch.sqrt(mse/len(preds))
    
    if train==True:
        return mse, rmse
    else:
        mae = torch.sum(abs_error)
        mape = torch.sum(abs_error/targets)

        targets_mean = torch.mean(targets)
        preds_mean = torch.mean(preds)
        r_square_numerator = torch.sum((targets-targets_mean)*(preds-preds_mean))**2
        r_square_denominator = torch.sum((targets-targets_mean)**2)*torch.sum((preds-preds_mean)**2)
        r_square = r_square_numerator / r_square_denominator


        esd = torch.sum((abs_error - torch.mean(abs_error))**2)
        return mse, rmse, mae, mape, r_square, esd

In [133]:
def calculate_scores(model, data_loader):
  criterion = Loss 
  train_examples = 0
  total_mae = total_mse = total_mape = total_r_square = total_esd = 0.0

  for batch, (features, targets) in enumerate(data_loader):
    preds = model(features)
    
    mse, rmse, mae, mape, r_square, esd = criterion(preds, targets, train=False)
    total_mae += mae
    total_mse += mse
    total_mape += mape
    train_examples +=len(features)
    total_r_square += r_square
    total_esd += esd

  total_RMSE_loss = format(torch.sqrt((total_mse)/train_examples), '.5f')
  total_mse_loss = format(total_mse/train_examples, '.5f')
  total_mae_loss = format(total_mae/train_examples, '.5f')
  total_mape_loss = format(total_mape/train_examples, '.5f')
  total_r_square = format(total_r_square, '.5f')
  total_esd = format(torch.sqrt(esd/train_examples), '.5f')
  

  print(f'total_RMSE_loss: {total_RMSE_loss}\ntotal_mse_loss: {total_mse_loss}\ntotal_mae_loss: {total_mae_loss}\ntotal_mape_loss: {total_mape_loss}\ntotal_r_square:{total_r_square}\ntotal_esd: {total_esd}')

# Training the Model Using Train Data

In [134]:
net.train()
criterion = Loss
EPOCHS= 2000        ####################################################
for epoch in range(EPOCHS):
  train_examples = 0
  train_loss = 0

  for batch ,(features, targets) in enumerate(train_dataloader):
    preds = net(features)
    # print(type(preds), preds.shape)
    mse_loss, rmse_loss = criterion(preds, targets, train=True)
    optimizer.zero_grad()
    rmse_loss.backward()
    optimizer.step()
    
    train_examples += len(features)
    train_loss += mse_loss
  
  train_loss = torch.sqrt(train_loss/train_examples)
  print(f'Epoch = {epoch+1} RMSE Training Loss = {format(train_loss, ".5f")}')

Epoch = 1 RMSE Training Loss = 8.96101
Epoch = 2 RMSE Training Loss = 8.71332
Epoch = 3 RMSE Training Loss = 8.27878
Epoch = 4 RMSE Training Loss = 7.55949
Epoch = 5 RMSE Training Loss = 6.45720
Epoch = 6 RMSE Training Loss = 4.86439
Epoch = 7 RMSE Training Loss = 2.69654
Epoch = 8 RMSE Training Loss = 0.71238
Epoch = 9 RMSE Training Loss = 1.58353
Epoch = 10 RMSE Training Loss = 1.00365
Epoch = 11 RMSE Training Loss = 0.89702
Epoch = 12 RMSE Training Loss = 0.73853
Epoch = 13 RMSE Training Loss = 0.62104
Epoch = 14 RMSE Training Loss = 0.60274
Epoch = 15 RMSE Training Loss = 0.54646
Epoch = 16 RMSE Training Loss = 0.49247
Epoch = 17 RMSE Training Loss = 0.55172
Epoch = 18 RMSE Training Loss = 0.54146
Epoch = 19 RMSE Training Loss = 0.52127
Epoch = 20 RMSE Training Loss = 0.52361
Epoch = 21 RMSE Training Loss = 0.52560
Epoch = 22 RMSE Training Loss = 0.51445
Epoch = 23 RMSE Training Loss = 0.51661
Epoch = 24 RMSE Training Loss = 0.52505
Epoch = 25 RMSE Training Loss = 0.52129
Epoch = 2

# Evaluating the Model Using Test Data

In [135]:
net.eval()
with torch.set_grad_enabled(False):
  calculate_scores(net,test_dataloader)

total_RMSE_loss: 0.62001
total_mse_loss: 0.38441
total_mae_loss: 0.50692
total_mape_loss: 0.06326
total_r_square:0.92556
total_esd: 0.35698


# -----------------------------------------------------------------------END--------------------------------------------------------------


